### 모듈 불러오기

In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
import warnings
import gc
from six.moves import urllib
warnings.filterwarnings('ignore')

In [49]:
# Scalers
from sklearn.preprocessing import StandardScaler
    # 입력된 1개 이상의 배열을 동일한 순서로 섞어준다.
from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline
    # 여러 모델 변환 개체의 결과를 연결한다.
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import LabelEncoder

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from scipy.stats import reciprocal, uniform

from sklearn.ensemble import AdaBoostClassifier

# Cross-validation
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_validate

# GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Common data processors
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils import check_array
from scipy import sparse

# Accuracy Score
from sklearn.metrics import accuracy_score

In [50]:
np.random.seed(123)

%matplotlib inline
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

In [51]:
train = pd.read_csv('./data/train_users_2.csv')
test = pd.read_csv('./data/test_users.csv')

In [52]:
id_test = test['id']
labels = train['country_destination'].values
df_train = train.drop(['country_destination'], axis=1)
#변수명 맘에 듬...train_flag
train_flag = df_train.shape[0]

In [53]:
train.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


In [54]:
test.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser
0,5uwns89zht,2014-07-01,20140701000006,NaN,FEMALE,35.0,facebook,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
1,jtl0dijy2j,2014-07-01,20140701000051,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
2,xx0ulgorjt,2014-07-01,20140701000148,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome
3,6c6puo6ix0,2014-07-01,20140701000215,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,IE
4,czqhjk3yfe,2014-07-01,20140701000305,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Safari


In [55]:
print(f'train columns is {len(train.columns)}', f'test columns is {len(test.columns)}')

train columns is 16 test columns is 15


In [56]:
print(f'train shape is {train.shape}',
      f'test shape is {test.shape}')


train shape is (213451, 16) test shape is (62096, 15)


In [57]:
set(train.columns) - set(test.columns)

{'country_destination'}

In [58]:
train['country_destination'].value_counts()

NDF      124543
US        62376
other     10094
FR         5023
IT         2835
GB         2324
ES         2249
CA         1428
DE         1061
NL          762
AU          539
PT          217
Name: country_destination, dtype: int64

In [59]:
train['country_destination'].value_counts(normalize=True)

NDF      0.583473
US       0.292226
other    0.047290
FR       0.023532
IT       0.013282
GB       0.010888
ES       0.010536
CA       0.006690
DE       0.004971
NL       0.003570
AU       0.002525
PT       0.001017
Name: country_destination, dtype: float64

In [60]:
train['country_destination'].nunique()

12

In [61]:
df_total = pd.concat([df_train, test], axis=0, ignore_index=True)

In [62]:
df_total.shape

(275547, 15)

In [63]:
# na값이 너무 많아서 빼는듯?!
df_total['date_first_booking'].isna().sum()

186639

In [64]:
df_total = df_total.drop(['id','date_first_booking'], axis=1)

In [65]:
# vstack <- 열의 개수가 같은 배열들을 세로로 합쳐준다.
date_ac = np.vstack(df_total['date_account_created'].astype(str).apply(lambda x:list(map(int, x.split('-')))).values)

In [66]:
#일자가 str형태라 split후 map을 사용하여 전부 int화
df_total['date_account_created'].astype(str).apply(
    lambda x: list(map(int, x.split('-'))))

0         [2010, 6, 28]
1         [2011, 5, 25]
2         [2010, 9, 28]
3         [2011, 12, 5]
4         [2010, 9, 14]
              ...      
275542    [2014, 9, 30]
275543    [2014, 9, 30]
275544    [2014, 9, 30]
275545    [2014, 9, 30]
275546    [2014, 9, 30]
Name: date_account_created, Length: 275547, dtype: object

In [67]:
df_total['Day'] = date_ac[:,0]
df_total['Month'] = date_ac[:,1]
df_total['year'] = date_ac[:,2]

df_total = df_total.drop(['date_account_created'], axis=1)

In [68]:
time_stp = np.vstack(df_total['timestamp_first_active'].astype(str).apply(lambda x: list(map(int, [x[:4],x[4:6],x[6:8],x[8:]]))).values)
time_stp

array([[  2009,      3,     19,  43255],
       [  2009,      5,     23, 174809],
       [  2009,      6,      9, 231247],
       ...,
       [  2014,      9,     30, 235408],
       [  2014,      9,     30, 235430],
       [  2014,      9,     30, 235901]])

In [69]:
df_total['tfa_day'] = time_stp[:,0]
df_total['tfa_Month'] = time_stp[:,1]
df_total['tfa_year'] = time_stp[:, 2]

df_total = df_total.drop(['timestamp_first_active'], axis=1)

In [70]:
df_total.loc[((df_total['age'] < 14) | (df_total['age'] > 100)),'age'] = -1

In [71]:
df_total['age'].max()

100.0

### OHE

In [74]:
class CategoricalEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, encoding='onehot', categories='auto', dtype=np.float64, handle_unknown='error'):
        self.encoding = encoding
        self.categories = categories
        self.dtype = dtype
        self.handle_unknown = handle_unknown
    
    def fit(self, X, y=None):
        """Fit the CategoricalEncoder to X.
        Parameters
        --------------
        X : array-like, shape [n_samples, n_feature]
            The data to determine the categories of each feature.
        Returns
        --------------
        self
        """

        if self.encoding not in ['onehot', 'onehot-dense', 'ordinal']:
            template = ("encoding should be either 'onehot', 'onehot-dense' or 'ordinal' , got %s")
            
            raise ValueError(template % self.handle_unknown)

        if self.handle_unknown not in ['error', 'ignore']:
            template = ("handle_unknown should be either 'error' or 'ignore', got %s")

            raise ValueError(template % self.handle_unknown)

        if self.encoding == 'ordinal' and self.handle_unknown == 'ignore':
            raise ValueError("handle_unknown='ignore' is not supported for encoding='ordinal'")

        X = check_array(X, dtype=np.object, accept_sparse='csc', copy=True)
        n_samples, n_features = X.shape

        self._label_encoders_ = [LabelEncoder() for _ in range(n_features)]

        for i in range(n_features):
            le = self._label_encoders_[i]
            # i번째 피쳐
            Xi = X[:, i]

            #카테고리 리스트를 설정 안 해줄 경우(auto)
            if self.categories == 'auto':
                le.fit(Xi)

            #카테고리 리스트 범주를 직접 설정해줄 경우
            else:
                # np.in1d(x,y) <- x배열이 y배열의 원소를 포함하고 있는지 여부를 불리언 배열로 반환 
                # 카테고리 리스트 범주가 데이터에 전부 유효한지 판단
                valid_mask = np.in1d(Xi, self.categories[i])
                if not np.all(valid_mask):
                    if self.handle_unknown == 'error':
                        diff = np.unique(Xi[~valid_mask])
                        msg = ("Found unknown categories {0} in column {1} during fit".format(diff, i))
                        raise ValueError(msg)
                le.classes_ = np.array(np.sort(self.categories[i]))
            
        self.categories_ = [le.classes_ for le in self._label_encoders_]

        return self

    def transform(self, X):
        """Transform X using one-hot encoding.
        Parameters
        ------------
        X : array-like, shape [n_samples, n_features]
            The data to encode.
        Returns
        ------------
        X_out : sparse matrix or a 2-d array
            Transformed input.
        """
        X = check_array(X, accept_sparse='csc', dtype=np.object, copy=True)
        n_samples, n_features = X.shape
        X_int = np.zeros_like(X, dtype=np.int)
        X_mask = np.ones_like(X, dtype=np.bool)

        for i in range(n_features):
            valid_mask = np.in1d(X[:, i], self.categories_[i])

            if not np.all(valid_mask):
                if self.handle_unknown == 'error':
                    diff = np.unique(X[~valid_mask, i])
                    msg = ("Found unknown categories {0} in column {1} during transform".format(diff, i))
                    raise ValueError(msg)

                else: 
                    X_mask[:, i] = valid_mask
                    X[:, i][~valid_mask] = self.categories_[i][0]
            X_int[:, i] = self._label_encoders_[i].transform(X[:,i])
        
        if self.encoding == 'ordinal':
            return X_int.astype(self.dtype, copy=False)

        mask = X_mask.ravel()
        n_values = [cats.shape[0] for cats in self.categories_]
        n_values = np.array([0] + n_values)
        indices = np.cumsum(n_values)

        column_indices = (X_int + indices[:-1]).ravel()[mask]
        row_indices = np.repeat(np.arange(n_samples, dtype=np.int32),
                                n_features)[mask]
        data = np.ones(n_samples * n_features)[mask]

        out = sparse.csc_matrix((data, (row_indices, column_indices)),
                                shape=(n_samples, indices[-1]),
                                dtype=self.dtype).tocsr()
        if self.encoding == 'onehot-dense':
            return out.toarray()
        else:
            return out

In [45]:
#입력받은 array에 대한 유효성 검사
check_array(np.array([[[1,1,1], [2,2,2], 3], [4, 5, 6],[7,8,9]]),
            dtype=np.object, accept_sparse='csc', copy=True)

array([[list([1, 1, 1]), list([2, 2, 2]), 3],
       [4, 5, 6],
       [7, 8, 9]], dtype=object)

In [41]:
#feature개수에 따른 labelencoder list 생성!
[LabelEncoder() for _ in range(5)]

[LabelEncoder(),
 LabelEncoder(),
 LabelEncoder(),
 LabelEncoder(),
 LabelEncoder()]

In [43]:
# np.in1d(x,y) <- x배열이 y배열의 원소를 포함하고 있는지 여부를 불리언 배열로 반환
x = np.array([1, 2, 3, 4, 5, 6])
y = np.array([2, 4])

np.in1d(x, y)


array([False,  True, False,  True, False, False])

In [75]:
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.attribute_names]

In [76]:
cat_pipeline = Pipeline([
    ("selector", DataFrameSelector(['affiliate_channel',
                                    'affiliate_provider', 'first_device_type'])),
    ("cat_encoder", CategoricalEncoder(encoding='onehot-dense')),
])

num_pipeline = Pipeline([
    ("selector", DataFrameSelector(
        ['signup_flow', 'Day', 'Month', 'year', 'tfa_day', 'tfa_Month', 'tfa_year'])),
    ('std_scaler', StandardScaler()),
])

full_pipeline = FeatureUnion(transformer_list=[
    ("cat_pipeline", cat_pipeline),
    ("num_pipeline", num_pipeline)

])


In [77]:
#create the dummies for the other categorical variables to apply transformation.

features = ['gender', 'language', 'signup_method',
            'first_affiliate_tracked', 'signup_app', 'first_browser']

for f in features:
    df_total_dummy = pd.get_dummies(df_total[f], prefix=f)
    df_total = df_total.drop([f], axis=1)
    df_total = pd.concat((df_total, df_total_dummy), axis=1)

# Now splitting train and test
x = df_total[:train_flag]
x_test = df_total[train_flag:]

In [78]:
final_train_X = full_pipeline.fit_transform(x)
final_test_X = full_pipeline.transform(x_test)
le = LabelEncoder()
train_set_y = le.fit_transform(labels)

In [79]:
forest_class = RandomForestClassifier(random_state=42)

n_estimators = [100, 500]
min_samples_split = [10, 20]

param_grid_forest = {'n_estimators': n_estimators,
                     'min_samples_split': min_samples_split}


rand_search_forest = GridSearchCV(forest_class, param_grid_forest, cv=4, refit=True,
                                  n_jobs=-1, verbose=2)

rand_search_forest.fit(final_train_X, train_set_y)


Fitting 4 folds for each of 4 candidates, totalling 16 fits


GridSearchCV(cv=4, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_samples_split': [10, 20],
                         'n_estimators': [100, 500]},
             verbose=2)

In [80]:
random_estimator = rand_search_forest.best_estimator_

y_pred_random_estimator = random_estimator.predict_proba(final_train_X)


In [81]:
y_pred = random_estimator.predict_proba(final_test_X)

# We take the 5 highest probabilities for each person
ids = []  # list of ids
cts = []  # list of countries
for i in range(len(id_test)):
    idx = id_test[i]
    ids += [idx] * 5
    cts += le.inverse_transform(np.argsort(y_pred[i])[::-1])[:5].tolist()

# Generating a csv file with the predictions
sub = pd.DataFrame(np.column_stack((ids, cts)), columns=['id', 'country'])
sub.to_csv('output_randomForest.csv', index=False)


In [82]:
sub


,id,country
0,5uwns89zht,NDF
1,5uwns89zht,US
2,5uwns89zht,FR
3,5uwns89zht,other
4,5uwns89zht,IT
...,...,...
310475,9uqfg8txu3,NDF
310476,9uqfg8txu3,US
310477,9uqfg8txu3,other
310478,9uqfg8txu3,CA
